In [1]:
O_min, O_max, z_min, z_max, u, d, alpha = var('O_min O_max z_min z_max u d alpha')

In [2]:
T_V(z_min, z) = function('T_V')(z_min, z)

In [3]:
T_L(z) = function('T_L')(z)

In [4]:
sigma_t(x) = function('sigma_t')(x)

In [5]:
def rendering_integral():
    return integral(sigma_t(z) * T_V(z_min, z) * T_L(z), z, z_min, var('z_max'), hold=True)

In [6]:
pretty_print(rendering_integral())

integrate(T_L(z)*T_V(z_min, z)*sigma_t(z), z, z_min, z_max)

## Basic rendering equation

In [7]:
def transmittance(f, lower_limit, upper_limit):
    return exp(-integral(f(x), x, lower_limit, upper_limit, hold=True))

In [8]:
extinction = var('sigma'); extinction

sigma

In [9]:
sigma_t(x) = extinction

In [10]:
T_V(z_min, z) = transmittance(sigma_t, z_min, z); pretty_print(T_V)

(z_min, z) |--> e^(-integrate(sigma, x, z_min, z))

In [11]:
T_L(z) = 1; pretty_print(T_L)

z |--> 1

In [12]:
pretty_print(rendering_integral().simplify_full())

(e^(sigma*z_max) - e^(sigma*z_min))*e^(-sigma*z_max)

## Rendering equation with self-shadowing

In [13]:
def light_ray_optical_thickness(z):
    return O_min + ((z - z_min) / (z_max - z_min)) * (O_max - O_min)

In [14]:
T_L(z) = exp(-light_ray_optical_thickness(z)); pretty_print(T_L)

z |--> e^(-O_min - (O_max - O_min)*(z - z_min)/(z_max - z_min))

In [15]:
pretty_print(rendering_integral().factor())

sigma*(z_max - z_min)*(e^(sigma*z_max + O_max) - e^(sigma*z_min + O_min))*e^(-sigma*z_max - O_max - O_min)/(sigma*z_max - sigma*z_min + O_max - O_min)

## Extinction falloff

In [16]:
falloff(l) = exp(-5 * l ^ 2 / u ^ 2)

In [17]:
#falloff(l) = 1 / (1 + (3 * l^2) / u); pretty_print(falloff)

In [18]:
def extinction(z, sigma):
    l = sqrt(d^2 + (z - z_min)^2 - 2 * d * (z-z_min) * cos(alpha))
    ext = sigma * falloff(l)    
    return ext

## Rendering equation with self-shadowing and extinction fading

In [19]:
pretty_print(extinction(z, var('sigma')))

sigma*e^(5*(2*d*(z - z_min)*cos(alpha) - d^2 - (z - z_min)^2)/u^2)

In [20]:
sigma_t(z) = extinction(z, var('sigma')); pretty_print(sigma_t)

z |--> sigma*e^(5*(2*d*(z - z_min)*cos(alpha) - d^2 - (z - z_min)^2)/u^2)

In [21]:
T_V(z_min, z) = transmittance(sigma_t, z_min, z); pretty_print(T_V)

(z_min, z) |--> e^(-integrate(sigma*e^(5*(2*d*(x - z_min)*cos(alpha) - d^2 - (x - z_min)^2)/u^2), x, z_min, z))

In [22]:
assume(z_max > z_min)

In [23]:
assume(u > 0)

In [24]:
assume(z_min <= z <= z_max)

In [25]:
assume(O_max*z_min-O_min*z_max + (O_min-O_max)*z>0)

In [26]:
assume(O_max - O_min + z_max - z_min > 0)

In [27]:
T_L(z) = exp(-light_ray_optical_thickness(z)); pretty_print(T_L)

z |--> e^(-O_min - (O_max - O_min)*(z - z_min)/(z_max - z_min))

In [28]:
#pretty_print(T_L(z).simplify())

In [29]:
final_equation = rendering_integral()

In [30]:
pretty_print(final_equation)

integrate(sigma*e^(-O_min - (O_max - O_min)*(z - z_min)/(z_max - z_min) + 5*(2*d*(z - z_min)*cos(alpha) - d^2 - (z - z_min)^2)/u^2 - integrate(sigma*e^(5*(2*d*(x - z_min)*cos(alpha) - d^2 - (x - z_min)^2)/u^2), x, z_min, z)), z, z_min, z_max)

In [31]:
pretty_print(T_V(z_min, z_max).simplify())

e^(-1/10*(sqrt(5)*sqrt(pi)*u*erf(sqrt(5)*d*cos(alpha)/u)*e^(5*d^2*cos(alpha)^2/u^2 - 5*d^2/u^2) - sqrt(5)*sqrt(pi)*u*erf(sqrt(5)*(d*cos(alpha) - z_max + z_min)/u)*e^(5*d^2*cos(alpha)^2/u^2 - 5*d^2/u^2))*sigma)

In [35]:
pretty_print(T_V(z_min, z).simplify())

e^(-1/10*(sqrt(5)*sqrt(pi)*u*erf(sqrt(5)*d*cos(alpha)/u)*e^(5*d^2*cos(alpha)^2/u^2 - 5*d^2/u^2) - sqrt(5)*sqrt(pi)*u*erf(sqrt(5)*(d*cos(alpha) - z + z_min)/u)*e^(5*d^2*cos(alpha)^2/u^2 - 5*d^2/u^2))*sigma)

In [32]:
assume((O_max*z_min-O_min*z_max + (O_min-O_max)*z)>0)

In [33]:
assumptions()

[z_max > z_min,
 u > 0,
 z_min <= z,
 -(O_max - O_min)*z - O_min*z_max + O_max*z_min > 0,
 O_max - O_min + z_max - z_min > 0]

In [34]:
#solved = final_equation.unhold().factor(); pretty_print(solved)